# Test DataIO Python Library (New) routines

This Jupyter notebook is a revised testing notebook aimed at testing the new pickle importing functions that
avoid the need to manipulate the strings in CSVs on import to convert them into the expected lists.

In [1]:
# This forces a reload of any external library file if it changes.  
# Useful when developing external libraries since otherwise Jupyter 
# will not re-import any library without restarting the python kernel.

%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import datetime

# Import COVID data IO routines from external python library
import COVIDlib.data_IO as COVID_IO

## Define variables of interest below
data_dir = 'our_data/'    # Data directory for the COVID datafiles
test_dir = 'test_data/'   # Data directory for storing test datafiles

## Define FIPS corresponding to various local areas
ClayFIPS = 27027
CassFIPS = 38017
MNFIPS = 27
NDFIPS = 38

# Test John Hopkins DataIO

Execute and test the John Hopkins DataIO routines first authored by Luke

In [3]:
# Trying the new routines to import dataframes from CSV with
# conversion of strings to lists.  Also allows a single 
# getLocalDataFrame() function for pulling to local data from
# ANY of our dataframes (since they all index based on FIPS).

# Retrieve John Hopkins dataframes
(JH_state_df, JH_cnty_df) = COVID_IO.PtoCDRDataFrames()
# Retrieve State-Level data for Minnesota
MN_df = COVID_IO.getLocalDataFrame(MNFIPS, JH_state_df)
# Retrieve County-level data for Clay County
CLAY_df = COVID_IO.getLocalDataFrame(ClayFIPS, JH_cnty_df)

# Retrieve Test John Hopkins dataframes
test_cntyfile = test_dir+"TEST_countylevel_combinedCDR.p"
test_statefile = test_dir+"TEST_statelevel_combinedCDR.p"
(testJH_state_df, testJH_cnty_df) = COVID_IO.PtoCDRDataFrames(countyFile = test_cntyfile, stateFile = test_statefile)
# Retrieve TEST State-Level data for Minnesota
TESTMN_df = COVID_IO.getLocalDataFrame(MNFIPS, testJH_state_df)
# Retrieve TEST County-level data for Clay County
TESTCLAY_df = COVID_IO.getLocalDataFrame(ClayFIPS, testJH_cnty_df)

In [4]:
print("Testing John Hopkins Test Data")
print(("-"*100))
if TESTMN_df['Dates'].values[0][0] == np.datetime64('2020-03-22') and TESTMN_df['Confirmed'].values[0][0] == 0:
    print("Correct Date format: {0} Compared to 2020-03-22 & Correct Death Format: {1} compared to 0".format(TESTMN_df['Dates'].values[0][0],TESTMN_df['Confirmed'].values[0][0]))
else:
    print("Incorrect Results")
if TESTCLAY_df['Dates'].values[0][0] == np.datetime64('2020-03-22') and TESTCLAY_df['Confirmed'].values[0][0] == 10:
    print("Correct Date format: {0} Compared to 2020-03-22 & Correct Death Format: {1} compared to 10".format(TESTCLAY_df['Dates'].values[0][0],TESTCLAY_df['Confirmed'].values[0][0]))
else:
    print("Incorrect Results")

Testing John Hopkins Test Data
----------------------------------------------------------------------------------------------------
Correct Date format: 2020-03-22 Compared to 2020-03-22 & Correct Death Format: 0 compared to 0
Correct Date format: 2020-03-22 Compared to 2020-03-22 & Correct Death Format: 10 compared to 10


In [5]:
##
## The Clay county data should have a steady 10 confirmed per day starting March 22, 2020, so the number of confirmed increases as 10 cases per day (means dConfirmed is 10/day)
## First death occurs 14 days later (since I "kill" 10% of the infected at the end of 14 days, the other 90% are recovered).  Check this!

##
## The Minnesota data should have an unreal situation of 10 additional new cases a day from March 22 to June 1.  We still "kill" 10% of the infected 14 days later and mark the
## other 90% "recovered."  Check this.

In [6]:
print("----------------------------------------\nTesting CDR Functions for JH Data\n----------------------------------------\n")
print("County Level:\n-------------\n")

testConfirmed = TESTCLAY_df['Confirmed'][0]
expectedConfirmed = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720]

if testConfirmed == expectedConfirmed:
    print("County Confirmed cases - Passed")
else:
    print("County Confirmed cases - Failed\n\tExpected: List w/ 10 new cases per day\tActual: {0}".format(testConfirmed))

testDeaths = TESTCLAY_df['Deaths'][0]
expectedDeaths = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58]

if testDeaths == expectedDeaths and testDeaths[14] == 1:
    print("County Deaths - Passed")
else:
    print("County Deaths - Failed\n\tExpected: List w/ 10% of infected patients dying, the first occuring 14 days in\tActual: {0}".format(testDeaths))
    
testRecovery = TESTCLAY_df['Recovered'][0]
expectedRecovery = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 18, 27, 36, 45, 54, 63, 72, 81, 90, 99, 108, 117, 126, 135, 144, 153, 162, 171, 180, 189, 198, 207, 216, 225, 234, 243, 252, 261, 270, 279, 288, 297, 306, 315, 324, 333, 342, 351, 360, 369, 378, 387, 396, 405, 414, 423, 432, 441, 450, 459, 468, 477, 486, 495, 504, 513, 522]

if testRecovery == expectedRecovery:
    print("County Recovered - Passed")
else:
    print("County Recovered - Failed\n\tExpected: List w/ 90% of infected patients having recovered\tActual: {0}".format(testRecovery))

print("\n")
countyDeaths = COVID_IO.GetCountyDeaths(testJH_cnty_df, ClayFIPS)
countyInfections = COVID_IO.GetCountyInfections(testJH_cnty_df, ClayFIPS)
countyRecovering = COVID_IO.GetCountyRecoveries(testJH_cnty_df, ClayFIPS)
if countyDeaths == 58:
    print("Display Most Recent Number of Deaths - Passed\n\tTotal COVID-19 Deaths: {0}".format(countyDeaths))
else:
    print("Display Most Recent Number of Deaths - Failed\n\tExpected COVID-19 Deaths: 58")
if countyInfections == 720:
    print("Display Most Recent Number of Infections - Passed\n\tTotal COVID-19 Infections: {0}".format(countyInfections))
else:
    print("Display Most Recent Number of Infections - Failed\n\tExpected COVID-19 Infections: 720")
if countyRecovering == 522:
    print("Display Most Recent Number of Recoveries - Passed\n\tTotal COVID-19 Recoveries: {0}".format(countyRecovering))
else:
    print("Display Most Recent Number of Recoveries - Failed\n\tExpected COVID-19 Recoveries: 720")
    
print("\nState Level:\n------------\n")
################################################################
testConfirmedState = TESTMN_df['Confirmed'][0]
expectedConfirmedState = [0, 10, 30, 60, 100, 150, 210, 280, 360, 450, 550, 660, 780, 910, 1050, 1200, 1360, 1530, 1710, 1900, 2100, 2310, 2530, 2760, 3000, 3250, 3510, 3780, 4060, 4350, 4650, 4960, 5280, 5610, 5950, 6300, 6660, 7030, 7410, 7800, 8200, 8610, 9030, 9460, 9900, 10350, 10810, 11280, 11760, 12250, 12750, 13260, 13780, 14310, 14850, 15400, 15960, 16530, 17110, 17700, 18300, 18910, 19530, 20160, 20800, 21450, 22110, 22780, 23460, 24150, 24850, 25560]

if testConfirmedState == expectedConfirmedState:
    print("State Confirmed cases - Passed")
else:
    print("State Confirmed cases - Failed\n\tExpected: List w/ 10 new cases per day\tActual: {0}".format(testConfirmed))

testDeathsState = TESTMN_df['Deaths'][0]
expectedDeathsState = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 6, 10, 15, 21, 28, 36, 45, 55, 66, 78, 91, 105, 120, 136, 153, 171, 190, 210, 231, 253, 276, 300, 325, 351, 378, 406, 435, 465, 496, 528, 561, 595, 630, 666, 703, 741, 780, 820, 861, 903, 946, 990, 1035, 1081, 1128, 1176, 1225, 1275, 1326, 1378, 1431, 1485, 1540, 1596, 1653]

if testDeathsState == expectedDeathsState and testDeathsState[15] == 1:
    print("State Deaths - Passed")
else:
    print("State Deaths - Failed\n\tExpected: List w/ 10% of infected patients dying, the first occuring 14 days in\tActual: {0}".format(testDeaths))
    
testRecoveryState = TESTMN_df['Recovered'][0]
expectedRecoveryState = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 27, 54, 90, 135, 189, 252, 324, 405, 495, 594, 702, 819, 945, 1080, 1224, 1377, 1539, 1710, 1890, 2079, 2277, 2484, 2700, 2925, 3159, 3402, 3654, 3915, 4185, 4464, 4752, 5049, 5355, 5670, 5994, 6327, 6669, 7020, 7380, 7749, 8127, 8514, 8910, 9315, 9729, 10152, 10584, 11025, 11475, 11934, 12402, 12879, 13365, 13860, 14364, 14877]
if testRecoveryState == expectedRecoveryState:
    print("State Recovered - Passed")
else:
    print("State Recovered - Failed\n\tExpected: List w/ 90% of infected patients having recovered\tActual: {0}".format(testRecovery))
    
    
print("\n")
stateDeaths = COVID_IO.GetStateDeaths(testJH_state_df, MNFIPS)
stateInfections = COVID_IO.GetStateInfections(testJH_state_df, MNFIPS)
stateRecovering = COVID_IO.GetStateRecoveries(testJH_state_df, MNFIPS)
if stateDeaths == 1653:
    print("Display Most Recent Number of Deaths - Passed\n\tTotal COVID-19 Deaths: {0}".format(stateDeaths))
else:
    print("Display Most Recent Number of Deaths - Failed\n\tExpected COVID-19 Deaths: 1653")
if stateInfections == 25560:
    print("Display Most Recent Number of Infections - Passed\n\tTotal COVID-19 Infections: {0}".format(stateInfections))
else:
    print("Display Most Recent Number of Infections - Failed\n\tExpected COVID-19 Infections: 25560")
if stateRecovering == 14877:
    print("Display Most Recent Number of Recoveries - Passed\n\tTotal COVID-19 Recoveries: {0}".format(stateRecovering))
else:
    print("Display Most Recent Number of Recoveries - Failed\n\tExpected COVID-19 Recoveries: 14877")

----------------------------------------
Testing CDR Functions for JH Data
----------------------------------------

County Level:
-------------

County Confirmed cases - Passed
County Deaths - Passed
County Recovered - Passed


Display Most Recent Number of Deaths - Passed
	Total COVID-19 Deaths: 58
Display Most Recent Number of Infections - Passed
	Total COVID-19 Infections: 720
Display Most Recent Number of Recoveries - Passed
	Total COVID-19 Recoveries: 522

State Level:
------------

State Confirmed cases - Passed
State Deaths - Passed
State Recovered - Passed


Display Most Recent Number of Deaths - Passed
	Total COVID-19 Deaths: 1653
Display Most Recent Number of Infections - Passed
	Total COVID-19 Infections: 25560
Display Most Recent Number of Recoveries - Passed
	Total COVID-19 Recoveries: 14877


## Test Apple and Google Mobility DataIO

Execute and test the Apple and Google Mobility DataIO routines first authored by Dio

In [7]:
# Trying the new routines to import dataframes from CSV with
# conversion of strings to lists.  Also allows a single 
# getLocalDataFrame() function for pulling to local data from
# ANY of our dataframes (since they all index based on FIPS).


## 
## Retrieve the Apple Mobility Data
##

# Retrieve Apple Mobility Dataframe
(aapl_cnty_df, aapl_state_df) = COVID_IO.PtoAAPLMobilityDataFrames()
# Get real Clay county and Minnesota mobility data
aapl_CLAY_df = COVID_IO.getLocalDataFrame(ClayFIPS, aapl_cnty_df)
aapl_MN_df = COVID_IO.getLocalDataFrame(MNFIPS, aapl_state_df)

# Retrieve TEST Apple Mobility Dataframe
test_cntyfile = test_dir+"TEST_aapl_mobility_cnty.p"
test_statefile = test_dir+"TEST_aapl_mobility_state.p"
(testaapl_cnty_df, testaapl_state_df) = COVID_IO.PtoAAPLMobilityDataFrames(countyFile = test_cntyfile, stateFile = test_statefile)
# Get TEST Clay county and Minnesota mobility data
testaapl_CLAY_df = COVID_IO.getLocalDataFrame(ClayFIPS, testaapl_cnty_df)
testaapl_MN_df = COVID_IO.getLocalDataFrame(MNFIPS, testaapl_state_df)

## 
## Retrieve the Google Mobility Data
##

# Retrieve Google Mobility Dataframe
(goog_cnty_df, goog_state_df) = COVID_IO.PtoGOOGMobilityDataFrames()
# Get real Clay county and Minnesota mobility data
goog_CLAY_df = COVID_IO.getLocalDataFrame(ClayFIPS, goog_cnty_df)
goog_MN_df = COVID_IO.getLocalDataFrame(MNFIPS, goog_state_df)

# Retrieve TEST Google Mobility Dataframe
test_cntyfile = test_dir+"TEST_goog_mobility_cnty.p"
test_statefile = test_dir+"TEST_goog_mobility_state.p"
(testgoog_cnty_df, testgoog_state_df) = COVID_IO.PtoGOOGMobilityDataFrames(countyFile = test_cntyfile, stateFile = test_statefile)
# Get TEST Clay county and Minnesota mobility data
testgoog_CLAY_df = COVID_IO.getLocalDataFrame(ClayFIPS, testgoog_cnty_df)
testgoog_MN_df = COVID_IO.getLocalDataFrame(MNFIPS, testgoog_state_df)

In [8]:
#expected Google Test Data output placed into variables
retRecPer = [0.0, 4.158, 8.135, 11.756, 14.863, 17.321, 19.021, 19.89, 19.89, 19.021, 17.321, 14.863, 11.756, 8.135, 4.158, 0.0, -4.158, -8.135, -11.756, -14.863, -17.321, -19.021, -19.89, -19.89, -19.021, -17.321, -14.863, -11.756, -8.135, -4.158, -0.0, 4.158, 8.135, 11.756, 14.863, 17.321, 19.021, 19.89, 19.89, 19.021, 17.321, 14.863, 11.756, 8.135, 4.158, 0.0, -4.158, -8.135, -11.756, -14.863, -17.321, -19.021, -19.89, -19.89, -19.021, -17.321, -14.863, -11.756, -8.135, -4.158, -0.0, 4.158, 8.135, 11.756, 14.863, 17.321, 19.021, 19.89, 19.89, 19.021, 17.321, 14.863, 11.756, 8.135, 4.158, 0.0, -4.158, -8.135, -11.756, -14.863, -17.321, -19.021, -19.89, -19.89, -19.021, -17.321, -14.863, -11.756, -8.135, -4.158, -0.0, 4.158, 8.135, 11.756, 14.863, 17.321, 19.021, 19.89, 19.89, 19.021]
googletestDates = ['2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29', '2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05', '2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09', '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13', '2020-03-14', '2020-03-15', '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-21', '2020-03-22', '2020-03-23', '2020-03-24', '2020-03-25', '2020-03-26', '2020-03-27', '2020-03-28', '2020-03-29', '2020-03-30', '2020-03-31', '2020-04-01', '2020-04-02', '2020-04-03', '2020-04-04', '2020-04-05', '2020-04-06', '2020-04-07', '2020-04-08', '2020-04-09', '2020-04-10', '2020-04-11', '2020-04-12', '2020-04-13', '2020-04-14', '2020-04-15', '2020-04-16', '2020-04-17', '2020-04-18', '2020-04-19', '2020-04-20', '2020-04-21', '2020-04-22', '2020-04-23', '2020-04-24', '2020-04-25', '2020-04-26', '2020-04-27', '2020-04-28', '2020-04-29', '2020-04-30', '2020-05-01', '2020-05-02', '2020-05-03', '2020-05-04', '2020-05-05', '2020-05-06', '2020-05-07', '2020-05-08', '2020-05-09', '2020-05-10', '2020-05-11', '2020-05-12', '2020-05-13', '2020-05-14', '2020-05-15', '2020-05-16', '2020-05-17', '2020-05-18', '2020-05-19', '2020-05-20', '2020-05-21', '2020-05-22', '2020-05-23', '2020-05-24']

groc = [0.0, 4.158, 8.135, 11.756, 14.863, 17.321, 19.021, 19.89, 19.89, 19.021, 17.321, 14.863, 11.756, 8.135, 4.158, 0.0, -4.158, -8.135, -11.756, -14.863, -17.321, -19.021, -19.89, -19.89, -19.021, -17.321, -14.863, -11.756, -8.135, -4.158, -0.0, 4.158, 8.135, 11.756, 14.863, 17.321, 19.021, 19.89, 19.89, 19.021, 17.321, 14.863, 11.756, 8.135, 4.158, 0.0, -4.158, -8.135, -11.756, -14.863, -17.321, -19.021, -19.89, -19.89, -19.021, -17.321, -14.863, -11.756, -8.135, -4.158, -0.0, 4.158, 8.135, 11.756, 14.863, 17.321, 19.021, 19.89, 19.89, 19.021, 17.321, 14.863, 11.756, 8.135, 4.158, 0.0, -4.158, -8.135, -11.756, -14.863, -17.321, -19.021, -19.89, -19.89, -19.021, -17.321, -14.863, -11.756, -8.135, -4.158, -0.0, 4.158, 8.135, 11.756, 14.863, 17.321, 19.021, 19.89, 19.89, 19.021]
par = [0.0, 2.079, 4.0675, 5.878, 7.4315, 8.6605, 9.5105, 9.945, 9.945, 9.5105, 8.6605, 7.4315, 5.878, 4.0675, 2.079, 0.0, -2.079, -4.0675, -5.878, -7.4315, -8.6605, -9.5105, -9.945, -9.945, -9.5105, -8.6605, -7.4315, -5.878, -4.0675, -2.079, -0.0, 2.079, 4.0675, 5.878, 7.4315, 8.6605, 9.5105, 9.945, 9.945, 9.5105, 8.6605, 7.4315, 5.878, 4.0675, 2.079, 0.0, -2.079, -4.0675, -5.878, -7.4315, -8.6605, -9.5105, -9.945, -9.945, -9.5105, -8.6605, -7.4315, -5.878, -4.0675, -2.079, -0.0, 2.079, 4.0675, 5.878, 7.4315, 8.6605, 9.5105, 9.945, 9.945, 9.5105, 8.6605, 7.4315, 5.878, 4.0675, 2.079, 0.0, -2.079, -4.0675, -5.878, -7.4315, -8.6605, -9.5105, -9.945, -9.945, -9.5105, -8.6605, -7.4315, -5.878, -4.0675, -2.079, -0.0, 2.079, 4.0675, 5.878, 7.4315, 8.6605, 9.5105, 9.945, 9.945, 9.5105]
#Transit percentages cannot be tested for since nan is null
#tra = [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
work = [0.0, 1.0395, 2.03375, 2.939, 3.71575, 4.33025, 4.75525, 4.9725, 4.9725, 4.75525, 4.33025, 3.71575, 2.939, 2.03375, 1.0395, 0.0, -1.0395, -2.03375, -2.939, -3.71575, -4.33025, -4.75525, -4.9725, -4.9725, -4.75525, -4.33025, -3.71575, -2.939, -2.03375, -1.0395, -0.0, 1.0395, 2.03375, 2.939, 3.71575, 4.33025, 4.75525, 4.9725, 4.9725, 4.75525, 4.33025, 3.71575, 2.939, 2.03375, 1.0395, 0.0, -1.0395, -2.03375, -2.939, -3.71575, -4.33025, -4.75525, -4.9725, -4.9725, -4.75525, -4.33025, -3.71575, -2.939, -2.03375, -1.0395, -0.0, 1.0395, 2.03375, 2.939, 3.71575, 4.33025, 4.75525, 4.9725, 4.9725, 4.75525, 4.33025, 3.71575, 2.939, 2.03375, 1.0395, 0.0, -1.0395, -2.03375, -2.939, -3.71575, -4.33025, -4.75525, -4.9725, -4.9725, -4.75525, -4.33025, -3.71575, -2.939, -2.03375, -1.0395, -0.0, 1.0395, 2.03375, 2.939, 3.71575, 4.33025, 4.75525, 4.9725, 4.9725, 4.75525]
res = [0.0, 1.0395, 2.03375, 2.939, 3.71575, 4.33025, 4.75525, 4.9725, 4.9725, 4.75525, 4.33025, 3.71575, 2.939, 2.03375, 1.0395, 0.0, -1.0395, -2.03375, -2.939, -3.71575, -4.33025, -4.75525, -4.9725, -4.9725, -4.75525, -4.33025, -3.71575, -2.939, -2.03375, -1.0395, -0.0, 1.0395, 2.03375, 2.939, 3.71575, 4.33025, 4.75525, 4.9725, 4.9725, 4.75525, 4.33025, 3.71575, 2.939, 2.03375, 1.0395, 0.0, -1.0395, -2.03375, -2.939, -3.71575, -4.33025, -4.75525, -4.9725, -4.9725, -4.75525, -4.33025, -3.71575, -2.939, -2.03375, -1.0395, -0.0, 1.0395, 2.03375, 2.939, 3.71575, 4.33025, 4.75525, 4.9725, 4.9725, 4.75525, 4.33025, 3.71575, 2.939, 2.03375, 1.0395, 0.0, -1.0395, -2.03375, -2.939, -3.71575, -4.33025, -4.75525, -4.9725, -4.9725, -4.75525, -4.33025, -3.71575, -2.939, -2.03375, -1.0395, -0.0, 1.0395, 2.03375, 2.939, 3.71575, 4.33025, 4.75525, 4.9725, 4.9725, 4.75525]

#expected Apple Test Daata ouput placed into Variables
applTestDates = ['2020-01-13', '2020-01-14', '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-18', '2020-01-19', '2020-01-20', '2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29', '2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05', '2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09', '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13', '2020-03-14', '2020-03-15', '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-21', '2020-03-22', '2020-03-23', '2020-03-24', '2020-03-25', '2020-03-26', '2020-03-27', '2020-03-28', '2020-03-29', '2020-03-30', '2020-03-31', '2020-04-01', '2020-04-02', '2020-04-03', '2020-04-04', '2020-04-05', '2020-04-06', '2020-04-07', '2020-04-08', '2020-04-09', '2020-04-10', '2020-04-11', '2020-04-12', '2020-04-13', '2020-04-14', '2020-04-15', '2020-04-16', '2020-04-17', '2020-04-18', '2020-04-19', '2020-04-20', '2020-04-21']
applDrivingMobility = [-90.0, -60.0, -30.0, 0.0, 30.0, 60.0, 90.0]

In [9]:
print("Testing google Mobility Test Data")
print(("-"*100))
if testgoog_CLAY_df['dates'].values[0][0] == np.datetime64(googletestDates[0]):
    print("pass Test Dates")
else:
    print("Error in Dates")
if testgoog_CLAY_df['retail_recreation_Percent'][0] == retRecPer:
    print("pass Test retail_recreation_Percent")
else:
    Print("Error in Retail & recreation percent")
if testgoog_CLAY_df['grocery_pharm_Percent'][0] == groc:
    print("pass Test grocery_pharm_Percent")
else:
    print("Error in Grocery_pharm_percent")
if testgoog_CLAY_df['parks_Percent'][0] == par:
    print("pass Test parks_Percent")
else:
    print("Error in Park_Percent")
if testgoog_CLAY_df['workplace_Percent'][0] == work:
    print("pass Test workplace_Percent")
else:
    print("Error in workplace_Percent")
if testgoog_CLAY_df['residential_Percent'][0] == res:
    print("pass Test residential_Percent")
else:
    print("Error in Residential_Percent")

Testing google Mobility Test Data
----------------------------------------------------------------------------------------------------
pass Test Dates
pass Test retail_recreation_Percent
pass Test grocery_pharm_Percent
pass Test parks_Percent
pass Test workplace_Percent
pass Test residential_Percent


In [10]:
print("Testing AppleMobility Test Data")
print(("-"*100))
if testaapl_CLAY_df['dates'].values[0][0] == np.datetime64(applTestDates[0]):
    print("pass Test Dates")
else:
    print("Error in Dates")
if testaapl_CLAY_df['driving_mobility'][0] == applDrivingMobility:
    print('pass Test driving_mobility')
else:
    print("Fail Test driving_mobility")

Testing AppleMobility Test Data
----------------------------------------------------------------------------------------------------
pass Test Dates
pass Test driving_mobility


In [11]:
##
## Test Apple Mobility Data should be checked
##  - For Clay county, the test data is a sawtooth pattern from -30 to +30 with a 7 day period starting on Feb. 15, 2020
##  - For Minnesota, the test data is a boxcar pattern 4 days at 30 followed by four days at -30 and back again starting on Feb. 15, 2020 starting on Feb. 15, 2020

In [12]:
##
## Test Google Mobility Data should be checked
##  - For Clay county, the test data is a sine wave with an amplitude of 20 and wavelength of 30 days starting on Feb. 15, 2020
##  - For Minnesota, the test data is a cosine wave with an amplitude of 20 and wavelength of 30 days starting on Feb. 15, 2020

## Test IMHE DataIO

Execute and test the Apple and Google Mobility DataIO routines first authored by Luke

In [13]:
# Trying the new routines to import dataframes from CSV with
# conversion of strings to lists.  Also allows a single 
# getLocalDataFrame() function for pulling to local data from
# ANY of our dataframes (since they all index based on FIPS).

## 
## Retrieve the IMHE Data
##

# Retrieve IMHE Dataframes
(summary_df, hospitalization_df) = COVID_IO.PtoIMHEDataFrames()
# Retrieve specific Dataframes and Data for MN
equip_df = COVID_IO.getLocalDataFrame(MNFIPS, summary_df)
icu_beds = COVID_IO.GetNumICUBeds(MNFIPS, summary_df)
all_beds = COVID_IO.GetNumAllBeds(MNFIPS, summary_df)
icu_usage = COVID_IO.GetICUBedUsage(MNFIPS, summary_df)
allbed_usage = COVID_IO.GetAllBedUsage(MNFIPS, summary_df)
hospital_df = COVID_IO.getLocalDataFrame(MNFIPS, hospitalization_df)

# Retrieve TEST IMHE Dataframes
testsummaryfile = test_dir+"TEST_imhe_summary.p"
testhospitalizationfile = test_dir+"TEST_imhe_hospitalizations.p"
(testsummary_df, testhospitalization_df) = COVID_IO.PtoIMHEDataFrames(summaryFile = testsummaryfile, hospitalFile = testhospitalizationfile)
# Retrieve specific TEST Dataframes and Data for MN
testequip_df = COVID_IO.getLocalDataFrame(MNFIPS, testsummary_df)
testequip_df = COVID_IO.GetEquipData(MNFIPS, testsummary_df)
testicu_beds = COVID_IO.GetNumICUBeds(MNFIPS, testsummary_df)
testall_beds = COVID_IO.GetNumAllBeds(MNFIPS, testsummary_df)
testicu_usage = COVID_IO.GetICUBedUsage(MNFIPS, testsummary_df)
testallbed_usage = COVID_IO.GetAllBedUsage(MNFIPS, testsummary_df)
testhospital_df = COVID_IO.getLocalDataFrame(MNFIPS, testhospitalization_df)

In [14]:
##
## Test IMHE data for Minnesota should be checked.
##  All summary dates were set to May 15, 2020 (lower May 1, upper June 15)
##  Bed capacity was set to 2000 and ICU capacity to 200. 
##  The test data assumes 100% usage.


In [15]:
# Testing Functions for IMHE Summary Data
print("----------------------------------------\nTesting Functions for IMHE Summary Data\n----------------------------------------\n")
if testicu_beds == 200:
    print("GetNumICUBeds : Passed")
else:
    print("GetNumICUBeds : Failed")
print("\tExpected value: 200 - Actual value: {0}".format(testicu_beds))
    
if testall_beds == 2000:
    print("GetNumAllBeds : Passed")
else:
    print("GetNumAllBeds : Failed")
print("\tExpected value: 2000 - Actual value: {0}".format(testall_beds) + "\n")
    
if testicu_usage == 200:
    print("GetICUBedUsage : Passed")
else:
    print("GetICUBedUsage : Failed")
print("\tExpected value: 200 - Actual value: {0}".format(testicu_usage))

if testallbed_usage == 2000:
    print("GetNumAllBeds : Passed")
else:
    print("GetNumAllBeds : Failed")
print("\tExpected value: 2000 - Actual value: {0}".format(testallbed_usage) + "\n")

if testequip_df['peak_bed_day_mean'].values[0] == np.datetime64('2020-05-15'):
    print("Peak Bed Day Mean - Passed")
else:
    print("Peak Bed Day Mean - Failed")

if testequip_df['peak_icu_bed_day_mean'].values[0] == np.datetime64('2020-05-15'):
    print("Peak ICU Bed Day Mean - Passed")
else:
    print("Peak ICU Bed Day Mean - Failed")
    
if testequip_df['peak_vent_day_mean'].values[0] == np.datetime64('2020-05-15'):
    print("Peak Vent Day Mean - Passed")
else:
    print("Peak Vent Day Mean - Failed")

----------------------------------------
Testing Functions for IMHE Summary Data
----------------------------------------

GetNumICUBeds : Passed
	Expected value: 200 - Actual value: 200
GetNumAllBeds : Passed
	Expected value: 2000 - Actual value: 2000

GetICUBedUsage : Passed
	Expected value: 200 - Actual value: 200
GetNumAllBeds : Passed
	Expected value: 2000 - Actual value: 2000

Peak Bed Day Mean - Passed
Peak ICU Bed Day Mean - Passed
Peak Vent Day Mean - Passed


In [16]:
##
## Test IMHE data for Minnesota should be checked.
##  All summary dates were set to May 15, 2020 (lower May 1, upper June 15)
##  Bed capacity was set to 2000 and ICU capacity to 200. 
##  The test data assumes 100% usage.

# The hospitalization data starts on March 22, 2020
# test_imhe_hospitalizations['allbed_mean'] is increasing at 1000 per day (10% margins of error on lower/upper)
# test_imhe_hospitalizations['ICUbed_mean'] is increasing at 100 per day (10% margins of error on lower/upper)
# test_imhe_hospitalizations['InvVen_mean'] is increasing at 50 per day (10% margins of error on lower/upper)
# For the hospitalization deaths/admits/ICU use, the test data has a constant 10 deaths/100 admits/20 ICU a day 
# (with 10% margins of error on lower/upper)

In [17]:
print("-----------------------------------------------\nTesting Functions for IMHE Hospitalization Data\n-----------------------------------------------\n")

if testhospital_df['dates'][0][0] == datetime.date(2020, 3, 22):
    print('Start date - Passed')
else:
    print('Start date - Failed')
    
print("\tExpected value: 2020-03-22 - Actual value: {0}\n".format(testhospital_df['dates'][0][0]))
    
allBedList = [0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000, 21000, 22000, 23000, 24000, 25000, 26000, 27000, 28000, 29000, 30000, 31000, 32000, 33000, 34000, 35000, 36000, 37000, 38000, 39000, 40000, 41000, 42000, 43000, 44000, 45000, 46000, 47000, 48000, 49000, 50000, 51000, 52000, 53000, 54000, 55000, 56000, 57000, 58000, 59000, 60000, 61000, 62000, 63000, 64000, 65000, 66000, 67000, 68000, 69000, 70000, 71000, 72000, 73000, 74000, 75000, 76000, 77000, 78000, 79000, 80000, 81000, 82000, 83000, 84000, 85000, 86000, 87000, 88000, 89000, 90000, 91000, 92000, 93000, 94000, 95000, 96000, 97000, 98000, 99000, 100000, 101000, 102000, 103000, 104000, 105000, 106000, 107000, 108000, 109000, 110000, 111000, 112000, 113000, 114000, 115000, 116000, 117000, 118000, 119000, 120000, 121000, 122000, 123000, 124000, 125000, 126000, 127000, 128000, 129000, 130000, 131000, 132000]
icuBedList = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600, 3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400, 4500, 4600, 4700, 4800, 4900, 5000, 5100, 5200, 5300, 5400, 5500, 5600, 5700, 5800, 5900, 6000, 6100, 6200, 6300, 6400, 6500, 6600, 6700, 6800, 6900, 7000, 7100, 7200, 7300, 7400, 7500, 7600, 7700, 7800, 7900, 8000, 8100, 8200, 8300, 8400, 8500, 8600, 8700, 8800, 8900, 9000, 9100, 9200, 9300, 9400, 9500, 9600, 9700, 9800, 9900, 10000, 10100, 10200, 10300, 10400, 10500, 10600, 10700, 10800, 10900, 11000, 11100, 11200, 11300, 11400, 11500, 11600, 11700, 11800, 11900, 12000, 12100, 12200, 12300, 12400, 12500, 12600, 12700, 12800, 12900, 13000, 13100, 13200]
ventList = [0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1050, 1100, 1150, 1200, 1250, 1300, 1350, 1400, 1450, 1500, 1550, 1600, 1650, 1700, 1750, 1800, 1850, 1900, 1950, 2000, 2050, 2100, 2150, 2200, 2250, 2300, 2350, 2400, 2450, 2500, 2550, 2600, 2650, 2700, 2750, 2800, 2850, 2900, 2950, 3000, 3050, 3100, 3150, 3200, 3250, 3300, 3350, 3400, 3450, 3500, 3550, 3600, 3650, 3700, 3750, 3800, 3850, 3900, 3950, 4000, 4050, 4100, 4150, 4200, 4250, 4300, 4350, 4400, 4450, 4500, 4550, 4600, 4650, 4700, 4750, 4800, 4850, 4900, 4950, 5000, 5050, 5100, 5150, 5200, 5250, 5300, 5350, 5400, 5450, 5500, 5550, 5600, 5650, 5700, 5750, 5800, 5850, 5900, 5950, 6000, 6050, 6100, 6150, 6200, 6250, 6300, 6350, 6400, 6450, 6500, 6550, 6600]

if testhospital_df['allbed_mean'][0] == allBedList:
    print("All bed data - Passed")
else:
    print("All bed data - Failed")

if testhospital_df['ICUbed_mean'][0] == icuBedList:
    print("ICU bed data - Passed")
else:
    print("ICU bed data - Failed")

if testhospital_df['InvVen_mean'][0] == ventList:
    print("Inv. Vent data - Passed")
else:
    print("Inv. Vent data - Failed")
    
print("\n")

tenCT = 0
deathsList = testhospital_df['deaths_mean'][0]
for entry in deathsList:
    if entry == 10:
        tenCT += 1
        
deathsLowerControl = [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
deathsUpperControl = [11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11]
deathsLower = testhospital_df['deaths_lower'][0]
deathsUpper = testhospital_df['deaths_upper'][0]

if tenCT == len(deathsList):
    print("Num. Deaths Mean - Passed")
else:
    print("Num. Deaths Mean - Failed") 
if deathsLowerControl == deathsLower:
    print("Num. Deaths Lower - Passed")
else:
    print("Num. Deaths Lower - Failed")
if deathsUpperControl == deathsUpper:
    print("Num. Deaths Upper - Passed")
else:
    print("Num. Deaths Upper - Failed")
print("\n")

tenCTAdmis = 0
admisList = testhospital_df['admis_mean'][0]
for entry in admisList:
    if entry == 10:
        tenCTAdmis += 1
admisLowerControl = [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
admisUpperControl = [11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11]
admisLower = testhospital_df['admis_lower'][0]
admisUpper = testhospital_df['admis_upper'][0]       

if tenCTAdmis == len(admisList):
    print("Num. Admissions Mean - Passed")
else:
    print("Num. Admissions Mean - Failed")
if admisLowerControl == admisLower:
    print("Num. Admissions Lower - Passed")
else:
    print("Num. Admissions Lower - Failed")
if admisUpperControl == admisUpper:
    print("Num. Admissions Upper - Passed")
else:
    print("Num. Admissions Upper - Failed")
print("\n")

tenCTicu = 0
icuList = testhospital_df['newICU_mean'][0]
for entry in icuList:
    if entry == 10:
        tenCTicu += 1

newICULowerControl = [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
newICUUpperControl = [11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11]
newICULower = testhospital_df['newICU_lower'][0]
newICUUpper = testhospital_df['newICU_upper'][0]         

if tenCTicu == len(deathsList):
    print("Num. ICU Mean - Passed")
else:
    print("Num. ICU Mean - Failed")
if newICULowerControl == newICULower:
    print("Num. ICU Lower - Passed")
else:
    print("Num. ICU Lower - Failed")
if newICUUpperControl == newICUUpper:
    print("Num. ICU Upper - Passed")
else:
    print("Num. ICU Upper - Failed")
print("\n")

-----------------------------------------------
Testing Functions for IMHE Hospitalization Data
-----------------------------------------------

Start date - Passed
	Expected value: 2020-03-22 - Actual value: 2020-03-22

All bed data - Passed
ICU bed data - Passed
Inv. Vent data - Passed


Num. Deaths Mean - Passed
Num. Deaths Lower - Passed
Num. Deaths Upper - Passed


Num. Admissions Mean - Passed
Num. Admissions Lower - Passed
Num. Admissions Upper - Passed


Num. ICU Mean - Passed
Num. ICU Lower - Passed
Num. ICU Upper - Passed




# Test $R_t$ Live Data


In [28]:
# Retrieve Test John Hopkins dataframes
test_statefile = test_dir+"TEST_Rt_live.p"
test_Rt_df = COVID_IO.PtoRtDataFrames(stateFile = test_statefile)
# Retrieve TEST State-Level data for Minnesota
TESTMN_df = COVID_IO.getLocalDataFrame(MNFIPS, test_Rt_df)

In [32]:
print("-----------------------------------------------\nTesting Functions for R_t Live Data\n-----------------------------------------------\n")

# Get indices for MN and ND
MNidx = TESTMN_df.index[TESTMN_df['FIPS'] == MNFIPS].tolist()[0]

if TESTMN_df.at[MNidx, 'dates'][0] == datetime.date(2020, 1, 19):
    print('Start date - Passed')
else:
    print('Start date - Failed')
    
print("\tExpected value: 2020-01-19 - Actual value: {0}\n".format(TESTMN_df.at[MNidx, 'dates'][0]))

# These are the expected values
dates_len = 100

# Assign bogus MN values
MN_mean = [0.9]*dates_len
MN_median = [0.9]*dates_len
MN_lower80 = [0.75]*dates_len
MN_upper80 = [1.05]*dates_len

if TESTMN_df.at[MNidx, 'Rt_mean'] == MN_mean:
    print("MN means - Passed")
else:
    print("MN means - Failed")

if TESTMN_df.at[MNidx, 'Rt_median'] == MN_median:
    print("MN Medians - Passed")
else:
    print("MN Medians - Failed")

if TESTMN_df.at[MNidx, 'Rt_lower_80'] == MN_lower80:
    print("MN lower_80 data - Passed")
else:
    print("MN lower_80 data - Failed")

if TESTMN_df.at[MNidx, 'Rt_upper_80'] == MN_upper80:
    print("MN upper_80 data - Passed")
else:
    print("MN upper_80 data - Failed")
    
print("\n")

-----------------------------------------------
Testing Functions for Rt Live Data
-----------------------------------------------

Start date - Passed
	Expected value: 2020-01-19 - Actual value: 2020-01-19

MN means - Passed
MN Medians - Passed
MN lower_80 data - Passed
MN upper_80 data - Passed


